## 2. Tensorflow Low Level API

### 2.1. 简介

本指南旨在指导您使用低级别TensorFlow API (TensorFlow Core)开始编程。您可以学习执行以下操作：

+ 管理TensorFlow程序([tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph))和TensorFlow runtime(程序运行时刻的状态)([tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session))，而不是依靠Estimator（高级API）来管理它们。
+ 使用[tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session)执行TensorFlow操作operation。
+ 在此低级别环境中使用高级别组件（[datasets](https://www.tensorflow.org/guide/low_level_intro?hl=en#datasets)、[layers](https://www.tensorflow.org/guide/low_level_intro?hl=en#layers)和[feature_columns](https://www.tensorflow.org/guide/low_level_intro?hl=en#feature_columns))。
+ 构建自己的训练循环，而不是使用[Estimator](https://www.tensorflow.org/guide/premade_estimators)提供的训练循环。

建议尽可能使用更高级API构建模型。以下是学习TensorFlow Core仍很重要的原因：

+ 直接使用低级TensorFlow操作，实验和调试都会更直接。
+ 在使用更高级API时，能够理解其内部工作原理。

**设置**

---

运行以下行来设置您的 Python 环境：


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf


**张量值Tensor Values**

---

TensorFlow中的核心数据单位是**张量tensor**。

+ 张量tensor是由任意维度的数组构成，数组元素是基本数据类型。张量tensor的阶rank是数组的维数。张量的形状shape是一个指定数组每个维度长度的整数元组。
+ 张量值tensor value: Tensorflow使用numpy数组表示。

以下是张量值tensor values的一些示例：

```
3. # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]
```

**Tensorflow低层API--Tensorflow Core演示**

---

可以认为TensorFlow Core程序由两个互相独立的部分组成：

1. 构建计算流图([tf.Graph](https://www.tensorflow.org/api_docs/python/tf/Graph?hl=zh-cn));
2. 运行计算流图([tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session?hl=zh-cn));

**计算流图Graph**

**计算流图**是一系列节点（操作[tf.Operation](https://www.tensorflow.org/api_docs/python/tf/Operation)）和边（张量[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)）的有向图，由两种类型对象组成：

+ [tf.Operation](https://www.tensorflow.org/api_docs/python/tf/Operation)(简称“op”)：图的节点。操作表示消耗和生成张量的数学计算；
+ [tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)：图的边。张量表示流经图的值。大多数TensorFlow函数会返回`tf.Tensors`。

> **tf.Tensors不具有值，它们只是计算流图中元素的句柄（智能指针or引用）。**

让我们来构建一个简单的计算流图。此计算流图中最基本的操作(op)是一个常量。构建操作(op)的Python函数`tf.constant()`将一个张量值tensor value作为输入参数值。生成的操作(op)不含有输入**值**。它在运行(`tf.Session`)时输出的是被传递给构造函数的值。我们可以创建如下所示的两个浮点数常量：


In [54]:
a = tf.constant(3. , dtype=tf.float32)
b = tf.constant(4.)  # default tf.float32 
total = a + b

print(a, a.op, a.graph, a.dtype, a.shape, a.value_index, sep='\n\n')
print('***' * 15)
print(b, b.op, b.graph, b.dtype, b.shape, b.value_index, sep='\n\n')
print(total)

Tensor("Const_12:0", shape=(), dtype=float32)

name: "Const_12"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 3.0
    }
  }
}



<dtype: 'float32'>

()

0
*********************************************
Tensor("Const_13:0", shape=(), dtype=float32)

name: "Const_13"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 4.0
    }
  }
}



<dtype: 'float32'>

()

0
Tensor("add_8:0", shape=(), dtype=float32)


请注意，打印张量并不会如您可能预期的那样打印输入的**值**`3.0`、`4.0`和`7.0`。上述语句只会构建计算流图。这些[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor?hl=zh-cn)对象仅表征将要运行的操作的结果。

计算流图中的每个操作(op)都拥有唯一的名称（示例输出的张量名称的"："之前的就是操作名）。这个名称不同于使用Python分配给相应对象的名称。张量是根据`生成它们的操作名称+输出索引`来命名，如上文的 "add:0" 所示。


**TensorBoard**

TensorFlow提供了一个名为TensorBoard的实用程序。TensorBoard的诸多功能之一是可视化计算流图。您只需要使用几个简单的命令就能轻松可视化计算流图。

首先将计算图保存为TensorBoard摘要文件Tensorflow summary file，具体操作如下所示：


In [4]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

这将在当前目录中生成一个`event`文件，其名称格式如下：

```
events.out.tfevents.{timestamp}.{hostname}
```

现在，在新的终端中使用以下 shell 命令启动 TensorBoard：

In [7]:
!tensorboard --logdir .

TensorBoard 1.12.0 at http://CharosendeMacBook-Air-2.local:6006 (Press CTRL+C to quit)
^C


接下来，在您的浏览器中打开TensorBoard的页面<http://charosendemacbook-air-2.local:6006/>，您应该会看到与以下图形类似的图：

![](https://www.tensorflow.org/images/getting_started_add.png?hl=zh-cn)

更多关于TensorBoard计算流图的可视化工具，请参阅[TensorBoard: Graph Visualization](https://www.tensorflow.org/guide/graph_viz)

**运行计算流图Session**

为了评估张量的值，需要实例化[tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session?hl=zh-cn)对象（非正式名称为**session**）。session会封装TensorFlow runtime的状态，并运行TensorFlow操作(op)。如果说[tf.Graph]()像一个`.py`源文件，那么[tf.Session]()就像一个`python`可执行对象。

下面的代码会创建一个[tf.Session](https://www.tensorflow.org/api_docs/python/tf/Session?hl=zh-cn)对象，然后调用其`run`方法来评估我们在上文中创建的`total`张量：

In [8]:
sess = tf.Session()
print(sess.run(total))


7.0


当您使用Session.run请求输出节点时，TensorFlow会回溯整个图，并流经为所请求的输出节点提供对应输入值的所有节点。因此此指令会打印预期的值 7.0。

**可以将多个张量传递给[tf.Session.run](https://www.tensorflow.org/api_docs/python/tf/Session?hl=zh-cn#run)。run方法以透明方式处理元组或字典的任何组合，返回的结果拥有相同的布局结构**，如下例所示：

In [9]:
print(sess.run({'ab': (a, b) , 'total': total}))

{'ab': (3.0, 4.0), 'total': 7.0}


**在调用[tf.Session.run](https://www.tensorflow.org/api_docs/python/tf/Session?hl=zh-cn#run)期间，任何[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor?hl=zh-cn)都只有单个值。**例如，以下代码调用[tf.random_uniform](https://www.tensorflow.org/api_docs/python/tf/random_uniform?hl=zh-cn)来生成一个[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor?hl=zh-cn)，后者会生成随机的三元素矢量（值位于`[0,1)`区间内）：

In [11]:
vec = tf.random_uniform(shape=(3, ))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))


[0.39608014 0.04684651 0.71877193]
[0.42865717 0.97670376 0.7490194 ]
(array([1.4137791, 1.933774 , 1.1837364], dtype=float32), array([2.4137793, 2.933774 , 2.1837363], dtype=float32))


每次调用`run`时，结果都会显示不同的随机值，但在单个`run`期间（`out1`和`out2`接收到相同的随机输入值），结果显示的值是一致的;

部分TensorFlow函数会返回`tf.Operations`，而不是`tf.Tensors`。对指令调用`run`的结果是`None`。Some TensorFlow functions return `tf.Operations` instead of `tf.Tensors`. The result of calling `run` on an Operation is `None`. You run an operation to cause a side-effect, not to retrieve a value. Examples of this include the [initialization](https://www.tensorflow.org/guide/low_level_intro?hl=en#Initializing%20Layers), and [training](https://www.tensorflow.org/guide/low_level_intro?hl=en#Training) ops demonstrated later.

**供给Feeding**

图可以参数化以便接受外部输入external input，也称为**占位符placeholder**。**占位符placeholder**表示承诺在稍后提供值，它就像函数参数, **可以使用tf.Session.run方法的feed_dict参数为占位符提供具体的值**。

In [12]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y
print(z)

Tensor("add_3:0", dtype=float32)


前面三行有点像函数。在这个函数中，我们定义了两个输入参数（`x`和`y`），以及对参数执行的操作(op)。我们可以使用[tf.Session.run](https://www.tensorflow.org/api_docs/python/tf/Session#run)方法的`feed_dict`参数为占位符提供具体的值，从而评估这个具有多个输入的图：

In [14]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))


7.5
[3. 7.]


另请注意，`feed_dict`参数可用于覆盖图中的任何张量。`placeholder`和其他`tf.Tensors`的唯一不同之处在于如果没有为占位符提供值，那么占位符会抛出错误。

**数据集**

---

占位符placeholder适用于简单的实验，而将数据传输到模型的首选方法是[tf.data](https://www.tensorflow.org/api_docs/python/tf/data)。

要从数据集中获取可运行的[tf.Tensor](https://www.tensorflow.org/api_docs/python/tf/Tensor)，您必须先将其转换成[tf.data.Iterator](https://www.tensorflow.org/api_docs/python/tf/data/Iterator)，然后调用迭代器的[tf.data.Iterator.get_next](https://www.tensorflow.org/api_docs/python/tf/data/Iterator#get_next)方法。

创建迭代器的最简单的方式是采用[make_one_shot_iterator](https://www.tensorflow.org/api_docs/python/tf/data/Dataset?hl=zh-cn#make_one_shot_iterator)方法。例如，在下面的代码中，`next_item`张量将在每次`run`调用时从`my_data`阵列返回一行：

In [24]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

到达数据流末端时，`Dataset`会抛出[OutOfRangeError](https://www.tensorflow.org/api_docs/python/tf/errors/OutOfRangeError?hl=zh-cn)。例如，下面的代码会一直读取`next_item`，直到没有数据可读：

In [25]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError as e:
        print('we captured one Error:', e)
        break

[0 1]
[2 3]
[4 5]
[6 7]
we captured one Error: End of sequence
	 [[node IteratorGetNext_3 (defined at <ipython-input-24-2e707051fb48>:8)  = IteratorGetNext[output_shapes=[[2]], output_types=[DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_3)]]

Caused by op 'IteratorGetNext_3', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/mac/.venvs/ml/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/mac/.venvs/ml/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/mac/.venvs/ml/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  Fil

要详细了解数据集和迭代器，请参阅[导入数据](https://www.tensorflow.org/guide/datasets)。

**层Layer**

---

可训练的模型必须修改图中的参数值，以便在输入相同值的情况下获得新的输出值。将可训练参数添加到图中的首选方法是[tf.layers](https://www.tensorflow.org/api_docs/python/tf/layers)。

层将变量和作用于它们的操作打包在一起。例如，[密集连接层densely-connected layer](https://developers.google.com/machine-learning/glossary/#fully_connected_layer)会对每个输出对应的所有输入执行加权和，并应用可选的激活函数[activation function](https://developers.google.com/machine-learning/glossary/#activation_function)。连接权重和偏置由层对象管理。

**创建层Creating Layers**

下面的代码会创建一个[tf.layers.Dense]()层，该层会接受一批输入矢量，并为每个矢量生成一个输出值。要将层应用于输入值，请将该层当做函数来调用。例如：

In [34]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
print(linear_model, type(linear_model))
y = linear_model(x)
print(y, type(y))

<tensorflow.python.layers.core.Dense object at 0x128bd69e8> <class 'tensorflow.python.layers.core.Dense'>
Tensor("dense_4/BiasAdd:0", shape=(?, 1), dtype=float32) <class 'tensorflow.python.framework.ops.Tensor'>


层会检查其输入数据，以确定其内部变量的size。因此，我们必须在这里设置`x`占位符的形状，以便层构建正确size的权重矩阵。

我们现在已经定义了输出值 y 的计算，在我们运行计算之前，还需要处理一个细节。


**初始化层Initializing Layers**


层包含的变量必须先**初始化**，然后才能使用。尽管可以单独初始化各个变量，但也可以轻松地初始化一个 TensorFlow 图中的所有变量（如下所示）:

In [39]:
init = tf.global_variables_initializer()
print(init, type(init))
sess.run(init)

name: "init_4"
op: "NoOp"
input: "^dense/kernel/Assign"
input: "^dense/bias/Assign"
input: "^dense_1/kernel/Assign"
input: "^dense_1/bias/Assign"
input: "^dense_2/kernel/Assign"
input: "^dense_2/bias/Assign"
input: "^dense_3/kernel/Assign"
input: "^dense_3/bias/Assign"
input: "^dense_4/kernel/Assign"
input: "^dense_4/bias/Assign"
 <class 'tensorflow.python.framework.ops.Operation'>


>>重要提示：调用[tf.global_variables_initializer](https://www.tensorflow.org/api_docs/python/tf/global_variables_initializer?hl=zh-cn)仅会创建并返回TensorFlow操作的句柄。当我们使用[tf.Session.run](https://www.tensorflow.org/api_docs/python/tf/Session?hl=zh-cn#run)运行该操作时，该操作将初始化所有全局变量。

另请注意，此`global_variables_initializer`仅会初始化创建初始化程序时图中就存在的变量。因此您应该在构建图表的最后一步添加初始化程序。

**执行层Executing Layers**

我们现在已经完成了层的初始化，可以像处理任何其他张量一样评估`linear_model`的输出张量了。例如，下面的代码：

In [40]:
print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

[[-2.350449 ]
 [-6.1733255]]


**Layer Function shortcut**

对于每个层类（如[tf.layers.Dense](https://www.tensorflow.org/api_docs/python/tf/layers/Dense))，TensorFlow 还提供了一个快捷函数（如[tf.layers.dense](https://www.tensorflow.org/api_docs/python/tf/layers/dense)）。两者唯一的区别是快捷函数版本是在单次调用中创建和运行层。例如，以下代码等同于较早的版本：

In [41]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))


[[0.35044795]
 [2.0934682 ]]


While convenient, this approach allows no access to the [tf.layers.Layer](https://www.tensorflow.org/api_docs/python/tf/layers/Layer) object. This makes introspection and debugging more difficult, and layer reuse impossible.尽管这种方法很方便，但无法访问 tf.layers.Layer 对象。这会让自省和调试变得更加困难，并且无法重复使用相同的层。

**特征列Feature columns**

(待补充)

**训练training**

---

您现在已经了解 TensorFlow core的基础知识了，我们来手动训练一个小型回归模型吧。

**定义数据**

我们首先来定义一些输入值 x，以及每个输入值的真实期望输出值 y_true：

In [42]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

**定义模型**

接下来，建立一个简单的线性模型，其输出值只有 1 个：

In [43]:
linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)

您可以如下评估预测值：

In [44]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))


[[-1.6602011]
 [-3.3204021]
 [-4.980603 ]
 [-6.6408043]]


该模型尚未接受训练，因此四个“预测”值并不理想。

**损失** 

要优化模型，您首先需要定义损失。我们将使用均方误差，这是回归问题的标准损失。

[tf.losses](https://www.tensorflow.org/api_docs/python/tf/losses?hl=zh-cn)模块提供了一系列常用的损失函数。您可以使用它来计算均方误差，具体操作如下所示：

In [46]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)
print(sess.run(loss))

7.569996


**训练**

TensorFlow 提供了使用标准优化算法的优化器[optimizer](https://developers.google.com/machine-learning/glossary/#optimizer)。这些优化器被实现为[tf.train.Optimizer](https://www.tensorflow.org/api_docs/python/tf/train/Optimizer?hl=zh-cn)的子类。它们会逐渐改变(迭代求解)每个变量，以便将损失最小化。最简单的优化算法是[梯度下降法](https://developers.google.com/machine-learning/glossary/?hl=zh-cn#gradient_descent),由[tf.train.GradientDescentOptimizer](https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer?hl=zh-cn)实现。它会根据损失相对于变量的导数大小来修改各个变量。例如：

In [48]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
print(optimizer, type(optimizer))
train = optimizer.minimize(loss)
print(train, type(train))

<tensorflow.python.training.gradient_descent.GradientDescentOptimizer object at 0x128f4d160> <class 'tensorflow.python.training.gradient_descent.GradientDescentOptimizer'>
name: "GradientDescent_1"
op: "NoOp"
input: "^GradientDescent_1/update_dense_6/kernel/ApplyGradientDescent"
input: "^GradientDescent_1/update_dense_6/bias/ApplyGradientDescent"
 <class 'tensorflow.python.framework.ops.Operation'>


该代码构建了优化所需的所有图组件，并返回一个训练操作。该训练指令在运行时会更新图中的变量。您可以按以下方式运行该操作：

In [49]:
for i in range(100):
    _, loss_value = sess.run((train, loss))
    print(loss_value)


7.569996
5.276876
3.6855836
2.5812767
1.814877
1.2829468
0.9137109
0.6573647
0.4793518
0.3556939
0.26975223
0.20998189
0.16837218
0.13936448
0.11910199
0.10490828
0.09492651
0.08786802
0.08283873
0.079218306
0.07657615
0.07461366
0.073123485
0.07196185
0.071028896
0.070255436
0.06959335
0.069009334
0.06848022
0.06798994
0.067527324
0.06708467
0.06665655
0.06623929
0.06583023
0.06542762
0.065030195
0.06463705
0.064247586
0.06386136
0.06347808
0.06309756
0.06271961
0.062344123
0.061971057
0.061600298
0.06123185
0.06086564
0.060501665
0.060139883
0.05978027
0.059422832
0.059067525
0.058714356
0.05836332
0.058014356
0.05766748
0.05732272
0.056979977
0.0566393
0.05630066
0.05596404
0.055629447
0.055296842
0.05496624
0.05463758
0.054310925
0.0539862
0.053663436
0.05334259
0.053023644
0.05270662
0.0523915
0.05207827
0.051766887
0.051457394
0.051149726
0.050843917
0.050539933
0.050237764
0.049937397
0.049638823
0.04934206
0.049047038
0.048753776
0.0484623
0.048172563
0.047884546
0.047598228
0.

**完整程序**

```
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

print(sess.run(y_pred))
```